In [1]:
!nvidia-smi
!pip -q install -U transformers datasets accelerate peft trl torch huggingface_hub gradio


Tue Dec 30 03:06:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   31C    P0             53W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
from huggingface_hub import login
login()  # paste your HF token


In [3]:
from datasets import load_dataset

DATASET_NAME = "bitext/Bitext-customer-support-llm-chatbot-training-dataset"
ds = load_dataset(DATASET_NAME)
print(ds)
print(ds["train"].column_names)
print(ds["train"][0])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Bitext_Sample_Customer_Support_Training_(…):   0%|          | 0.00/19.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26872 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['flags', 'instruction', 'category', 'intent', 'response'],
        num_rows: 26872
    })
})
['flags', 'instruction', 'category', 'intent', 'response']
{'flags': 'B', 'instruction': 'question about cancelling order {{Order Number}}', 'category': 'ORDER', 'intent': 'cancel_order', 'response': "I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you."}


In [4]:
def to_text(ex):
    cols = ex.keys()

    # common Bitext pattern
    if "instruction" in cols and "response" in cols:
        prompt, resp = ex["instruction"], ex["response"]
        return {"text": f"### Instruction:\n{prompt}\n\n### Response:\n{resp}"}

    # fallback patterns
    for p_key in ["prompt","query","question","customer_message","text","input"]:
        for r_key in ["response","answer","assistant","output","completion","agent_message"]:
            if p_key in cols and r_key in cols:
                return {"text": f"### Instruction:\n{ex[p_key]}\n\n### Response:\n{ex[r_key]}"}

    return {"text": str(ex)}

train = ds["train"].shuffle(seed=42)
split = train.train_test_split(test_size=0.02, seed=42)

train_ds = split["train"].map(to_text, remove_columns=split["train"].column_names)
eval_ds  = split["test"].map(to_text, remove_columns=split["test"].column_names)

print(train_ds[0]["text"][:400])


Map:   0%|          | 0/26334 [00:00<?, ? examples/s]

Map:   0%|          | 0/538 [00:00<?, ? examples/s]

### Instruction:
I need assistance tracking my reimbursement

### Response:
I'll get right on it! I completely understand your need for assistance in tracking your reimbursement. Rest assured, I'm here to help you every step of the way. Please provide me with the necessary details, such as the refund reference number or any other information related to your reimbursement. This will allow me to inv


In [7]:
!pip -q install -U trl transformers accelerate datasets peft

In [8]:
import trl, transformers
print("trl:", trl.__version__)
print("transformers:", transformers.__version__)


trl: 0.26.2
transformers: 4.57.3


In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, SFTConfig

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
OUT_DIR = "outputs/sft"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)

cfg = SFTConfig(
    output_dir=OUT_DIR,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=2e-5,
    logging_steps=25,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    bf16=torch.cuda.is_available(),
    report_to="none",
    dataset_text_field="text",
    max_length=512,
)

trainer = SFTTrainer(
    model=model,
    args=cfg,
    train_dataset=train_ds,     # must contain a "text" column
    eval_dataset=eval_ds,       # must contain a "text" column

    processing_class=tokenizer, # <-- key change for TRL 0.26.2
)

trainer.train()
trainer.save_model(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)

print("Saved to:", OUT_DIR)


Adding EOS to train dataset:   0%|          | 0/26334 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/26334 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/26334 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/538 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/538 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/538 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
200,0.720200,0.708385,0.740788,518521.000000,0.788084
400,0.653100,0.654410,0.681294,1037538.000000,0.798919
600,0.647700,0.630937,0.664509,1561296.000000,0.804282
800,0.612500,0.617655,0.641094,2087482.000000,0.806935
1000,0.612200,0.609255,0.639588,2618219.000000,0.809139
1200,0.608900,0.603947,0.632358,3150139.000000,0.810543
1400,0.595500,0.601581,0.628254,3681683.000000,0.811323
1600,0.594400,0.601103,0.627877,4210085.000000,0.811749


Saved to: outputs/sft


In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM

LOCAL = "outputs/sft"
HF_REPO = "BekiTila/bitext-sft-tinyllama-support"

tok = AutoTokenizer.from_pretrained(LOCAL)
mdl = AutoModelForCausalLM.from_pretrained(LOCAL)

tok.push_to_hub(HF_REPO)
mdl.push_to_hub(HF_REPO)

print("Pushed to:", HF_REPO)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...pgxovk0jg/tokenizer.model: 100%|##########|  500kB /  500kB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...v3wvujq/model.safetensors:   0%|          | 1.19MB / 4.40GB            

Pushed to: BekiTila/bitext-sft-tinyllama-support
